### Modules

In [8]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd

### Pre-define parameters

In [2]:
driver_path = 'D:/Users/figohjs/Documents/WebScraping/chromedriver.exe'
website = "https://www.sc.com.my/regulation/enforcement/have-you-seen-these-people"

### Web driver

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('headless')   
# options.add_argument("window-size=1200x600")
# driver = webdriver.Chrome(executable_path = driver_path, options = options)
driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)
driver.get(website)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [D:\Users\figohjs\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


### Udf

In [4]:
def extractInfo(text):
    tempDict = {}
    infoTextList = text.split('\n')

    #extract name
    name = re.search("(.*)\'s", infoTextList[0], re.I).group(1)
    tempDict['Name'] = name

    #extract ic
    nric = infoTextList[2]
    tempDict['NRIC'] = nric

    #track last element
    lastElementList = [re.search("(.*please contact Securities Commission Malaysia at:)", i).group(1) for i in infoTextList 
                       if re.search("please contact Securities Commision Malaysia", i)]
    if len(lastElementList)>0:
        lastElement = lastElementList[0]
        notImportantIndex = infoTextList.index(lastElement)
    else:
        notImportantIndex = -1

    #if business location and residential address are in popout-content
    if re.search('Last Known Business', ''.join(infoTextList)) \
    and re.search('Last Known Residential Address', ''.join(infoTextList)):
        businessLocIndex = infoTextList.index('Last Known Business / Employment address:')
        residentLocIndex = infoTextList.index('Last Known Residential Address:')
        #extract business location
        businessLoc = ' '.join(infoTextList[businessLocIndex + 1:residentLocIndex])
        #extract residential location
        residentLoc = ' '.join(infoTextList[residentLocIndex + 1:notImportantIndex])

    #if business location is not in popout-content
    if not re.search('Last Known Business', ''.join(infoTextList)) \
    and re.search('Last Known Residential Address', ''.join(infoTextList)):
        residentLocIndex = infoTextList.index('Last Known Residential Address:')
        #extract business location
        businessLoc = ''
        #extract residential location
        residentLoc = ' '.join(infoTextList[residentLocIndex + 1:notImportantIndex])    

    #clean up
    businessLoc = re.sub("ADDRESS [12] ", '', businessLoc)
    tempDict['BusinessLoc'] = businessLoc

    #clean up
    residentLoc = re.sub("ADDRESS [12] ", '', residentLoc)
    tempDict['ResidentLoc'] = residentLoc
    
    return tempDict

### Start

In [7]:
#Persons Wanted by the SC and List of People Sought to Assist in Investigation
className = "atom-235e66ed-8b22-4a94-a258-017e3fc3abad"
oneDiv =  driver.find_elements_by_xpath("//div[contains(@class, '%s')]//div"%className)

#get class name for both section
twoClassNames = [i.get_attribute("class") for i in oneDiv if re.search("row row", i.get_attribute("class"))]

#wanted list
#all class for a tag
xPath = "//div[@class = '%s']//a"%twoClassNames[0]
allClassAFirstSec = [i.get_attribute("class") for i in driver.find_elements_by_xpath(xPath)]

#all src attributes for img tag
xPath = "//div[@class = '%s']//img[@class = 'so-thumbnail']"%twoClassNames[0]
allImgFirstSec = driver.find_elements_by_xpath(xPath)
allImgSrcFirstSec = [i.get_attribute('src') for i in allImgFirstSec]

#store result
infoList = []
for no, imageSrc in enumerate(allImgSrcFirstSec):
#     print(imageSrc)
    driver.get(website)
    
    xPath1 = "//a[@class = '%s']/img[@src = '%s']"%(allClassAFirstSec[no], imageSrc)
    wait = WebDriverWait(driver, 10)
    # auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, xPath1)))
    auto_element = wait.until(EC.element_to_be_clickable((By.XPATH, xPath1)))
    # auto_element.click()
    driver.execute_script("arguments[0].click();", auto_element)
    
    xPath2 = "//div[contains(@class, 'mfp-content')]"
    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException)
    element = WebDriverWait(driver, 20, ignored_exceptions = ignored_exceptions)\
                            .until(EC.presence_of_element_located((By.XPATH, xPath2)))
    info = element.find_element_by_xpath(xPath2)
    result = extractInfo(info.text)
    infoList.append(result)
    
#sought to assist
#all class for a tag
xPath = "//div[@class = '%s']//a"%twoClassNames[1]
allClassASecondSec = [i.get_attribute("class") for i in driver.find_elements_by_xpath(xPath)]

#all src attributes for img tag
xPath = "//div[@class = '%s']//img[@class = 'so-thumbnail']"%twoClassNames[1]
allImgSecondSec = driver.find_elements_by_xpath(xPath)
allImgSrcSecondSec = [i.get_attribute('src') for i in allImgSecondSec]

infoList2 = []
for no, imageSrc in enumerate(allImgSrcSecondSec):
#     print(imageSrc)
    driver.get(website)
    
    xPath1 = "//a[@class = '%s']/img[@src = '%s']"%(allClassASecondSec[no], imageSrc)
    wait = WebDriverWait(driver, 10)
    # auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, xPath1)))
    auto_element = wait.until(EC.element_to_be_clickable((By.XPATH, xPath1)))
    # auto_element.click()
    driver.execute_script("arguments[0].click();", auto_element)
    
    xPath2 = "//div[contains(@class, 'mfp-content')]"
    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException)
    element = WebDriverWait(driver, 20, ignored_exceptions = ignored_exceptions)\
                            .until(EC.presence_of_element_located((By.XPATH, xPath2)))
    info = element.find_element_by_xpath(xPath2)
    result = extractInfo(info.text)
    infoList2.append(result)

In [9]:
dfWanted = pd.DataFrame(infoList).drop_duplicates()
dfWanted['Category'] = "WantedList"

dfAssist = pd.DataFrame(infoList).drop_duplicates()
dfAssist['Category'] = "SoughtToAssist"

In [10]:
dfWanted

,Name,NRIC,BusinessLoc,ResidentLoc,Category
0,ALICE POH GAIK LYE,550608-07-5550,N/A,"7 Jalan SS7/3, Kelana Jaya 47301 Petaling Jaya...",WantedList
2,GOH BAK MING,431127-10-5569,N/A,3 Jalan 16/14 Section 16 46350 Petaling Jaya S...,WantedList
4,WONG SHEE KAI (RICKY),811212-14-5957,"Asia Media Group Berhad No. 35, First Floor, J...","No. 5, Jalan Enak 2, Taman Gembira, 58200 Kual...",WantedList
6,UZIR BIN ABDUL SAMAD,720824-01-5543,"UAS Bistari Management Sdn Bhd, No. 11A, Jalan...","No. 14, Jalan SS 17/2A, 47500 Subang Jaya, Sel...",WantedList
8,TAN WHAI OON / JIGME PHENDE,600910-07-5001,"SJ Asset Management Sdn Bhd, Suite 12-10 & 12-...","No. 1, Jalan 16/6, Seksyen 16, 46350 Petaling ...",WantedList
10,"WONG JIT KIANG, DANIEL",551227-01-6359 (new) / 4905358 (old),,"9A-B, Impiana Condo, Jalan Ulu Klang, 68000 Am...",WantedList
12,YEOH GUAN YONG,761125-07-5011,Remisier’s Assistant of Hwang DBS Securities S...,"No. 556-A, Jln. Abdullah Ariff, 11400 Ayer Ita...",WantedList
14,"YEO WEE SIONG, GEORGE",S7022197G (Singapore),"Megan Media Holdings Berhad, No.4 Jalan SS13/5...","806 Bedok Resevoir Road, #0402, Singapore 4792...",WantedList
16,JENNIFER TAY SIEW KIEM,6353775,"S.K.Tay & Co. Suite, 19-04-01 Aras 4, Plaza Ke...","116-19-8, Jalan Sultan Abdul Samad, Brickfield...",WantedList


In [11]:
dfAssist

,Name,NRIC,BusinessLoc,ResidentLoc,Category
0,ALICE POH GAIK LYE,550608-07-5550,N/A,"7 Jalan SS7/3, Kelana Jaya 47301 Petaling Jaya...",SoughtToAssist
2,GOH BAK MING,431127-10-5569,N/A,3 Jalan 16/14 Section 16 46350 Petaling Jaya S...,SoughtToAssist
4,WONG SHEE KAI (RICKY),811212-14-5957,"Asia Media Group Berhad No. 35, First Floor, J...","No. 5, Jalan Enak 2, Taman Gembira, 58200 Kual...",SoughtToAssist
6,UZIR BIN ABDUL SAMAD,720824-01-5543,"UAS Bistari Management Sdn Bhd, No. 11A, Jalan...","No. 14, Jalan SS 17/2A, 47500 Subang Jaya, Sel...",SoughtToAssist
8,TAN WHAI OON / JIGME PHENDE,600910-07-5001,"SJ Asset Management Sdn Bhd, Suite 12-10 & 12-...","No. 1, Jalan 16/6, Seksyen 16, 46350 Petaling ...",SoughtToAssist
10,"WONG JIT KIANG, DANIEL",551227-01-6359 (new) / 4905358 (old),,"9A-B, Impiana Condo, Jalan Ulu Klang, 68000 Am...",SoughtToAssist
12,YEOH GUAN YONG,761125-07-5011,Remisier’s Assistant of Hwang DBS Securities S...,"No. 556-A, Jln. Abdullah Ariff, 11400 Ayer Ita...",SoughtToAssist
14,"YEO WEE SIONG, GEORGE",S7022197G (Singapore),"Megan Media Holdings Berhad, No.4 Jalan SS13/5...","806 Bedok Resevoir Road, #0402, Singapore 4792...",SoughtToAssist
16,JENNIFER TAY SIEW KIEM,6353775,"S.K.Tay & Co. Suite, 19-04-01 Aras 4, Plaza Ke...","116-19-8, Jalan Sultan Abdul Samad, Brickfield...",SoughtToAssist


In [ ]:
#Combine table
dfResult = pd.concat([dfWanted, dfAssist]).copy()
csvFile = 'D:/Users/figohjs/Documents/WebScraping/sc/2021-02-24_SC_wantedList.csv'
dfResult.to_csv(csvFile, index = False)